In [1]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
### YOUR CODE HERE
from tensorflow.keras import regularizers
###
import tensorflow.keras.utils as ku 
import numpy as np
import keras
from keras import layers
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
tokenizer = Tokenizer()
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt \
    -O /tmp/sonnets.txt
data = open('/tmp/sonnets.txt').read()

corpus = data.lower().split("\n")


tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

# create input sequences using list of tokens
input_sequences = []
for line in corpus:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)

--2021-02-10 20:33:03--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.193.128, 173.194.217.128, 172.217.204.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.193.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93578 (91K) [text/plain]
Saving to: ‘/tmp/sonnets.txt’

/tmp/sonnets.txt    100%[===================>]  91.38K  --.-KB/s    in 0.001s  

2021-02-10 20:33:03 (78.6 MB/s) - ‘/tmp/sonnets.txt’ saved [93578/93578]



In [3]:
# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

label = ku.to_categorical(label, num_classes=total_words)

In [10]:

model = keras.models.Sequential()
model.add(Embedding(total_words, 300, input_length=max_sequence_len-1))
model.add(layers.Bidirectional(layers.LSTM(512, recurrent_dropout=0.3, recurrent_regularizer=l2(0.01),return_sequences=True)))
model.add(layers.LSTM(256))
model.add(layers.Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics='accuracy')


In [11]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 10, 300)           963300    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 10, 1024)          3330048   
_________________________________________________________________
lstm_7 (LSTM)                (None, 256)               1311744   
_________________________________________________________________
dense_3 (Dense)              (None, 3211)              825227    
Total params: 6,430,319
Trainable params: 6,430,319
Non-trainable params: 0
_________________________________________________________________


In [12]:
address="./kaggle/working/weights-improvement.hdf5"
stop = EarlyStopping(monitor = 'val_accuracy', min_delta = 0, 
                             patience = 5, verbose = 1, mode = 'auto')
save = ModelCheckpoint(address, monitor = 'val_accuracy', 
                               verbose = 0, save_best_only = True)
callbacks = [stop, save]

history = model.fit(predictors, label, validation_split=0.20, epochs=50, verbose=1, callbacks = callbacks)

Epoch 1/50
387/387 [==============================] - 35s 90ms/step - loss: 7.2038 - accuracy: 0.0267 - val_loss: 6.7779 - val_accuracy: 0.0265
Epoch 2/50
387/387 [==============================] - 33s 87ms/step - loss: 6.3385 - accuracy: 0.0393 - val_loss: 6.8523 - val_accuracy: 0.0252
Epoch 3/50
387/387 [==============================] - 34s 87ms/step - loss: 6.0843 - accuracy: 0.0483 - val_loss: 6.9549 - val_accuracy: 0.0375
Epoch 4/50
387/387 [==============================] - 33s 86ms/step - loss: 5.7916 - accuracy: 0.0580 - val_loss: 7.0830 - val_accuracy: 0.0459
Epoch 5/50
387/387 [==============================] - 33s 86ms/step - loss: 5.4554 - accuracy: 0.0716 - val_loss: 7.2374 - val_accuracy: 0.0430
Epoch 6/50
387/387 [==============================] - 34s 87ms/step - loss: 5.0898 - accuracy: 0.0917 - val_loss: 7.3632 - val_accuracy: 0.0514
Epoch 7/50
387/387 [==============================] - 33s 85ms/step - loss: 4.6955 - accuracy: 0.1189 - val_loss: 7.5587 - val_accuracy:

Adding recurrent_dropout=0.3 and recurrent_regularizer=l2(0.01) did not improve the model's performance. It could have been inadvertently damaging the performance by randomly dropping the recurrent state, i.e., the output from the previous layers.